## Installing Dependencies

In [6]:
! pip install pandas
! pip install openpyxl

In [7]:
import pandas as pd

## Load Excel Data

In [8]:
csod_file_path = 'csod_schema.xlsx'
test_data_file_path = 'test_data.xlsx'

csod_data = pd.ExcelFile('/workspaces/DataAutomation/csod_schema.xlsx')
test_data = pd.ExcelFile('/workspaces/DataAutomation/TEST DATA.xlsx')

## Parsing the Schema using CarreFour as our First Example

In [9]:
csod_df = csod_data
test_df = test_data

## Definfing Column Mapping

In [ ]:
csod_df = csod_data.parse('Sheet1')
test_df = test_data.parse('CARREFOUR')


#schema and Test data 
column_mapping = {
    'year': 'Code', 
    'TOTAL QNTY': 'U_ProductDes',
    'TOTAL VALUE': 'U_StoreName',
    
    #todo: add more mappings here
}


## Setting Up the Database, and SQL statements

In [ ]:
table_name = 'test_table'

def generate_sql_statements(df, column_mapping, table_name):
    insert_statement = []
    for_, row in df.iterrows():
        #create a list of mapped Values
        values = [
            row['year'],
            row['TOTAL QNTY'],
            row['TOTAL VALUE']
        ]
        
        #Format Values for SQL compatibility
    formatted_values = [f"'{value}'" if isinstance(value, str) else value for value in values]
    
    sql_statement = f"INSERT INTO {table_name} (Code, U_ProductDes, U_StoreName) VALUES ({formatted_values[0]}, {formatted_values[1]}, {formatted_values[2]});"
    instert_statement.append(sql_statement)
    
    return insert_statement

OUTPUT SQL STATEMENTS TO A .SQL FILE

In [ ]:
with open('insert.sql', 'w') as f:
    for statement in generate_sql_statements:
        f.write(f"{statement}\n")
print ("SQL insert statements have been written to insert_statement.sql")